In [67]:
from llama_index.core import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    StorageContext,
    load_index_from_storage,
)
from llama_index.core.tools import QueryEngineTool, FunctionTool
from llama_index.core.query_engine import SubQuestionQueryEngine
from llama_index.core.agent import ReActAgent
from llama_index.llms.openai import OpenAI
import asyncio
import os
from dotenv import load_dotenv

In [19]:
# Load environment variables
load_dotenv()

True

In [20]:
# Define storage directory
storage_dir = "../.index_storage"

# Check if index already exists
if os.path.exists(storage_dir):
    print("Loading existing index...")
    storage_context = StorageContext.from_defaults(persist_dir=storage_dir)
    index = load_index_from_storage(storage_context)
else:
    print("Creating new index...")
    documents = SimpleDirectoryReader("../data").load_data()
    index = VectorStoreIndex.from_documents(documents)
    index.storage_context.persist(persist_dir=storage_dir)

Loading existing index...
Loading llama_index.core.storage.kvstore.simple_kvstore from ../.index_storage/docstore.json.
Loading llama_index.core.storage.kvstore.simple_kvstore from ../.index_storage/index_store.json.


/Users/user/Desktop/Python/rag-book/.venv/lib/python3.13/site-packages/typing_inspect.py:221: RuntimeWarning: coroutine 'main' was never awaited
  def is_union_type(tp):


In [79]:
query_engine_tools = [
    QueryEngineTool.from_defaults(
        query_engine=index.as_query_engine(),
        name="document_search",
        description="Luôn luôn sử dụng công cụ này để tìm kiếm thông tin trong sách giáo khoa cấp trung học phổ thông.",
    )
]

query_engine = SubQuestionQueryEngine.from_defaults(
    query_engine_tools=query_engine_tools,
    llm=OpenAI(model="gpt-4o-mini")
)

In [74]:
# Create the ReActAgent with the defined tools
agent = ReActAgent.from_tools(
    query_engine_tools,
    llm=OpenAI(model="gpt-4o-mini"),
    verbose=True,  # Set to True to see the agent's thought process (reasoning, tool calls),
    system_prompt=
    """
    Bạn là một Trợ lý Trích xuất và Tạo sinh (RAG) chuyên nghiệp, có nhiệm vụ cung cấp câu trả lời chính xác và đầy đủ cho các câu hỏi của người dùng, chỉ dựa trên thông tin được trích xuất từ các sách giáo khoa cấp trung học phổ thông.
    Hướng dẫn hoạt động:
    - Suy nghĩ: Luôn luôn suy nghĩ bằng tiếng Việt và trình bày rõ ràng quá trình suy nghĩ của bạn trước khi đưa ra câu trả lời cuối cùng.
    - Nguồn thông tin duy nhất: Mọi câu trả lời của bạn phải LUÔN LUÔN dựa vào các đoạn văn bản được trích xuất từ sách giáo khoa cấp trung học phổ thông đã được cung cấp. Tuyệt đối không sử dụng kiến thức tổng quát, thông tin bên ngoài, hoặc suy diễn cá nhân.
    - Độ chính xác và khách quan: Đảm bảo tính chính xác và khách quan của thông tin. Tránh thêm thắt, phóng đại, hoặc đưa ra ý kiến chủ quan.
    - Rõ ràng và súc tích: Trình bày câu trả lời một cách rõ ràng, súc tích, dễ hiểu và phù hợp với cấp độ kiến thức của học sinh trung học phổ thông.
    - Xử lý thông tin không có sẵn: Nếu bạn không tìm thấy thông tin cần thiết để trả lời câu hỏi trong các tài liệu được cung cấp, hãy thông báo rõ ràng rằng: "Tôi không tìm thấy thông tin này trong các sách giáo khoa cấp trung học phổ thông được cung cấp."
    - Ngôn ngữ: Luôn sử dụng tiếng Việt chuẩn và ngữ pháp chính xác.
    """,
)

In [75]:
# Now we can ask questions about the documents or do calculations
response = agent.chat(
    """
    Cuộc Cách mạng công nghiệp lần thứ tư được bắt đầu từ thời gian nào?

A. Từ những năm đầu thế kỉ XXI.

B. Từ những năm đầu thế kỉ XX.

C. Từ những năm cuối thế kỉ XX.

D. Từ những năm đầu thế kỉ XIX.
    """
)
print(response)

> Running step 27f64afc-66df-4a6e-8aa8-07828b53d790. Step input: 
    Cuộc Cách mạng công nghiệp lần thứ tư được bắt đầu từ thời gian nào?

A. Từ những năm đầu thế kỉ XXI.

B. Từ những năm đầu thế kỉ XX.

C. Từ những năm cuối thế kỉ XX.

D. Từ những năm đầu thế kỉ XIX.
    
Thought: The current language of the user is: Vietnamese. I need to use a tool to help me answer the question.
Action: document_search
Action Input: {'input': 'Cuộc Cách mạng công nghiệp lần thứ tư bắt đầu từ thời gian nào?'}
Observation: Cuộc Cách mạng công nghiệp lần thứ tư bắt đầu từ những năm đầu tiên của thế kỉ XXI.
> Running step d65ec6d2-b36a-42a2-b4b3-8f0616bf276f. Step input: None
Thought: I can answer without using any more tools. I'll use the user's language to answer.
Answer: A. Từ những năm đầu thế kỉ XXI.
A. Từ những năm đầu thế kỉ XXI.


In [ ]:
query_engine.query(
    """
    Các cuộc cách mạng công nghiệp thời kì hiện đại đem lại tác động tiêu cực nào sau đây về văn hóa?

    A. Xuất hiện nguy cơ đánh mất bản sắc văn hóa dân tộc.
    B. Khiến quá trình giao lưu văn hóa trở nên dễ dàng.
    C. Giúp việc tìm kiếm, chia sẻ thông tin trở nên thuận tiện.
    D. Giúp con người làm nhiều công việc bằng hình thức từ xa.
    """
)

Generated 3 sub questions.
[document_search] Q: Cuộc cách mạng công nghiệp hiện đại đã dẫn đến sự phân hóa trong lực lượng lao động như thế nào?
[document_search] Q: Có những nghiên cứu nào chỉ ra rằng cuộc cách mạng công nghiệp hiện đại khiến nhiều người lao động đối diện với nguy cơ mất việc làm?
[document_search] Q: Cuộc cách mạng công nghiệp hiện đại đã làm gia tăng sự xung đột giữa văn hóa truyền thống và hiện đại ra sao?
[document_search] A: Có những nghiên cứu chỉ ra rằng cuộc cách mạng công nghiệp hiện đại khiến nhiều người lao động đối diện với nguy cơ mất việc làm do sự tự động hoá giúp giải phóng sức lao động của con người, đặc biệt trong những công việc nguy hiểm, trong môi trường độc hại.
[document_search] A: Cuộc cách mạng công nghiệp hiện đại đã dẫn đến sự phân hóa trong lực lượng lao động bằng cách tăng số lượng người lao động có kỹ năng và trình độ chuyên môn cao, đồng thời giảm dần số lượng người lao động phổ thông.
[document_search] A: Cuộc cách mạng công nghiệp hiện

Response(response='C. Khiến nhiều người lao động đối diện với nguy cơ mất việc làm.', source_nodes=[NodeWithScore(node=TextNode(id_='e6bd7632-dc18-44bc-8931-59ef87c91add', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text='Sub question: Cuộc cách mạng công nghiệp hiện đại đã dẫn đến sự phân hóa trong lực lượng lao động như thế nào?\nResponse: Cuộc cách mạng công nghiệp hiện đại đã dẫn đến sự phân hóa trong lực lượng lao động bằng cách tăng số lượng người lao động có kỹ năng và trình độ chuyên môn cao, đồng thời giảm dần số lượng người lao động phổ thông.', mimetype='text/plain', start_char_idx=None, end_char_idx=None, metadata_seperator='\n', text_template='{metadata_str}\n\n{content}'), score=None), NodeWithScore(node=TextNode(id_='f91207e8-81b5-48a5-b445-6d468d949ded', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[]